# Project Oxford: Face API


In [2]:
import time 
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter
import httplib, urllib, base64 #General API Usage

In [3]:
# Variables

_url_detect = 'https://api.projectoxford.ai/face/v1.0/detect'
_url_verify = 'https://api.projectoxford.ai/face/v1.0/verify'
_url_group = 'https://api.projectoxford.ai/face/v1.0/group'
_key = '0b8afa8909154432bafeed6a3217fcde' #Here you have to paste your primary key
_maxNumRetries = 10

## 1. Face Detection (To get faceId)

In [4]:
def detectionRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url_detect, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result

In [5]:
def renderResultOnImage( result, img ):
    
    """Display the obtained results onto the input image"""

    for currFace in result:
        faceRectangle = currFace['faceRectangle']
        cv2.rectangle( img,(faceRectangle['left'],faceRectangle['top']),
                           (faceRectangle['left']+faceRectangle['width'], faceRectangle['top'] + faceRectangle['height']),
                       color = (255,0,0), thickness = 1 )

        faceLandmarks = currFace['faceLandmarks']

        for _, currLandmark in faceLandmarks.items():
            cv2.circle( img, (int(currLandmark['x']),int(currLandmark['y'])), color = (0,255,0), thickness= -1, radius = 1 )

    for currFace in result:
        faceRectangle = currFace['faceRectangle']
        faceAttributes = currFace['faceAttributes']

        textToWrite = "%c (%d)" % ( 'M' if faceAttributes['gender']=='male' else 'F', faceAttributes['age'] )
        cv2.putText( img, textToWrite, (faceRectangle['left'],faceRectangle['top']-15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1 )

##   Face detection

폴더 내에 있는 페이스 아이디 저장하기: test_faceId

In [27]:
import os

# Face detection parameters
params = { 'returnFaceAttributes': 'age,gender', 
           'returnFaceLandmarks': 'true'} 

headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/octet-stream'

json = None

#cur_dir = '/Users/robot/git_ryan/MS_Face_API/image/Park/park_etc/'
cur_dir = '/home/ryan/git_ryan/MS_Face_API/Taehee/'
test_file = os.listdir(cur_dir)

test_faceId = [] #face id info
file_name = [] #directory file name
#data_name = [] #data directory

for i in xrange(0, len(test_file)):
    file_name.append(cur_dir + test_file[i])
    
    with open (file_name[i], 'rb') as f:
        data = f.read()
        #data_name.append(data)

    try:
        result = detectionRequest( json, data, headers, params )
        test_faceId.append( result[0]['faceId'])
        time.sleep(1)
        print("success")
    except:
        print("fail filename: %s" %file_name[i])

In [28]:
len(test_file)
len(test_faceId)
test_faceId[0]
test_file[0]

'kim_291.jpg'

In [33]:
face_data = dict(zip(test_file, test_faceId))
print(face_data)

파일 저장하기

In [34]:
import cPickle as pickle

pickle.dump( face_data, open( "faceId.p", "wb"))

face_id = pickle.load( open ( "faceId.p", "rb"))
print(face_id)

## Face Data 출력하기

In [ ]:
#TODO 수정필요

if result[0] is not None:
    # Load the original image from disk
    data8uint = np.fromstring( data[0], np.uint8 ) # Convert string to an unsigned int array
    img = cv2.cvtColor( cv2.imdecode( data8uint, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )

    renderResultOnImage( result[0], img )

    ig, ax = plt.subplots(figsize=(15, 20))
    ax.imshow( img )

## 2. Face Grouping (Need faceId)

In [35]:
def groupRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url_group, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result

In [36]:
test_faceId

[u'912b2dc9-a6d4-4720-85ba-514de6842213',
 u'fdfbc246-de7f-47ce-a5d5-2e3136952e55',
 u'3f22cc91-28b4-4e92-b9f6-d86cf9966fb2',
 u'f5fbc6fc-9918-4bdf-b528-e2b95babccfd',
 u'79d9961d-8638-46ee-a2d8-f44486c51949',
 u'ad59537a-28c5-4426-bbb0-c6d81087cc15']

In [37]:
params = None

headers = dict()

headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json'

data = None

json = { 'faceIds': test_faceId } 

result = groupRequest( json, data, headers, params )
print(result['groups'])